In [43]:
import collections
import glob
import json
import os
from pprint import pprint
from sys_config import EXP_DIR
import warnings
warnings.filterwarnings('ignore')


import matplotlib.pyplot as plt
from matplotlib import cm
import pandas as pd
import seaborn as sns
import numpy as np

sns.set_style("white")
sns.set_context("notebook")
plt.rcParams['font.family'] = "serif"

from IPython.core.display import display, HTML


# %matplotlib inline
# %matplotlib notebook
# %matplotlib widget
%matplotlib ipympl


In [44]:
experiments = glob.glob(EXP_DIR + f"/experiments/efficiency/final.trans*/**")
experiments = glob.glob(EXP_DIR + f"/experiments/main/final.trans*/**") + experiments

In [45]:
def flatten(d, parent_key='', sep='.'):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)


def read_bleu(exp, file):
    with open(os.path.join(exp, file)) as json_file:
        data = json_file.read()
        data = data[:data.rfind("}")+1]
        bleu = flatten(json.loads(data), "bleu")
        return bleu

results = []
for exp in sorted(experiments):
    try:
        name = exp.split(os.sep)[-2]
        log = flatten(json.load(open(os.path.join(exp, f"{name}.json"))))
        bleu = read_bleu(exp, "BLEU.json")
        record = {**log, **bleu}
        record["pair"] = name.split("_")[0].split(".")[-1]
        record["model"] = name.split(".")[-1]
            
        results.append(record)
    except Exception as e:
        print(e)
    
#     for size in ["3M", "30M"]:
    for size in ["3M"]:
        try:
            name = exp.split(os.sep)[-2]
            pair = name.split("_")[0].split(".")[-1]
            model = name.split(".")[-1] + f"_fusion_{size}"
            log = flatten(json.load(open(os.path.join(exp, f"{name}.json"))))
            bleu = read_bleu(exp, f"BLEU_shallow_0.1_lm_news_{pair[2:]}_{size}_trans_big_best.json")

            record = {**log, **bleu}
            record["pair"] = pair
            record["model"] = model
            results.append(record)
        except Exception as e:
#             print(e)
            pass
                
df = pd.DataFrame.from_records(results)
df.head()

[Errno 2] No such file or directory: '/home/christos/PycharmProjects/seq3mt/experiments/experiments/main/final.trans.deen_postnorm_ls/20-04-13_09:13:03_21895060/BLEU.json'
[Errno 2] No such file or directory: '/home/christos/PycharmProjects/seq3mt/experiments/experiments/main/final.trans.entr_postnorm_ls/20-04-13_09:13:03_21895091/BLEU.json'
[Errno 2] No such file or directory: '/home/christos/PycharmProjects/seq3mt/experiments/experiments/main/final.trans.tren_postnorm_ls/20-04-13_09:13:03_21895080/BLEU.json'


,bleu.test.1,bleu.test.5,bleu.val.1,bleu.val.5,config.batch_tokens,config.config,config.cores,config.data.backtranslate_path,config.data.lowercase,config.data.prior_path,...,metrics.epoch_ppl.VALID - MT,metrics.loss,metrics.loss.MT,metrics.loss.PRIOR,metrics.lr,metrics.ppl,metrics.ppl.MT,model,name,pair
0,5.2,5.6,4.7,4.9,10000,../../configs/transformer/trans.deen_base....,4,/rds/project/t2_vol4/rds-t2-cs055/cbaziotis/wo...,True,None,...,NaN,"[8.979499459266663, 8.647571130232377, 8.44131...",NaN,NaN,"[6.176323555016366e-08, 3.149925013058347e-06,...","[7938.657499410571, 5696.294360598452, 4634.65...",NaN,deen_base_10000,final.trans.deen_base_10000,deen
1,5.3,5.8,4.7,5.0,10000,../../configs/transformer/trans.deen_base....,4,/rds/project/t2_vol4/rds-t2-cs055/cbaziotis/wo...,True,None,...,NaN,"[8.979499459266663, 8.647571130232377, 8.44131...",NaN,NaN,"[6.176323555016366e-08, 3.149925013058347e-06,...","[7938.657499410571, 5696.294360598452, 4634.65...",NaN,deen_base_10000_fusion_3M,final.trans.deen_base_10000,deen
2,5.0,5.3,4.4,4.7,10000,../../configs/transformer/trans.deen_base....,4,NaN,True,None,...,NaN,"[8.962945063908895, 8.629665634848855, 8.42027...",NaN,NaN,"[6.176323555016366e-08, 3.149925013058347e-06,...","[7808.319633393253, 5595.207097738077, 4538.15...",NaN,deen_base_10000,final.trans.deen_base_10000,deen
3,5.2,5.4,4.4,4.8,10000,../../configs/transformer/trans.deen_base....,4,NaN,True,None,...,NaN,"[8.962945063908895, 8.629665634848855, 8.42027...",NaN,NaN,"[6.176323555016366e-08, 3.149925013058347e-06,...","[7808.319633393253, 5595.207097738077, 4538.15...",NaN,deen_base_10000_fusion_3M,final.trans.deen_base_10000,deen
4,5.2,5.4,4.6,4.8,10000,../../configs/transformer/trans.deen_base....,4,NaN,True,None,...,NaN,"[8.999773820241293, 8.658453161066229, 8.45190...",NaN,NaN,"[6.176323555016366e-08, 3.149925013058347e-06,...","[8101.2513812581155, 5758.620111799558, 4684.0...",NaN,deen_base_10000,final.trans.deen_base_10000,deen


In [46]:
# drop the rest
# keep_cols = ["bleu.test.1", "bleu.test.5", "bleu.test.10", "bleu.val.1", "bleu.val.5", "bleu.val.10", "name", "pair"]
# keep_cols = ["bleu.test.5", "bleu.test.10", "bleu.val.5", "bleu.val.10", "model", "pair"]
keep_cols = ["bleu.test.5", "bleu.val.5", "model", "pair"]
keep = [k for k in df.columns if k in keep_cols]
df = df[keep]
df.rename(columns={'bleu.test.5': 'test', 'bleu.val.5': 'dev'}, inplace=True)
df.head()

,test,dev,model,pair
0,5.6,4.9,deen_base_10000,deen
1,5.8,5.0,deen_base_10000_fusion_3M,deen
2,5.3,4.7,deen_base_10000,deen
3,5.4,4.8,deen_base_10000_fusion_3M,deen
4,5.4,4.8,deen_base_10000,deen


In [47]:
df = df[df.columns[::-1]]


In [109]:
_std = lambda x: np.std(x, ddof=0)
def eval_pair(pair):
    _data = df[~df.model.str.contains("00")]
#     _data = _data[_data.model.str.contains("fusion")]
#     _data = df[df.model.str.contains("3M_kl_ls")]
    _data = _data[_data["pair"]==pair].groupby(["model"], as_index=False).agg([np.mean, _std]).sort_values(by="model", ascending=True).round(1)
    display(HTML(_data.style.bar(color='#d65f5f').render()))

In [110]:
eval_pair("deen")

In [111]:
eval_pair("ende")

In [112]:
eval_pair("tren")

In [113]:
eval_pair("entr")

In [201]:
plt.style.use('classic')
from matplotlib import rc

font_size = 11
nice_fonts = {
        # Use LaTeX to write all text
        "text.usetex": True,
        "font.family": "serif",
        # Use 10pt font in plots, to match 10pt font in document
        "axes.labelsize": font_size,
        "font.weight": "bold",
        "axes.labelweight": "bold",
        "font.size": font_size,
        # Make the legend/label fonts a little smaller
        "legend.fontsize": font_size,
        "xtick.labelsize": font_size,
        "ytick.labelsize": font_size,
}
rc('text', usetex=True)
sns.set_style("white", {
    "font.family": "serif",
    "font.serif": ["CMU Serif", "Times", "serif"],
})
plt.rcParams.update(nice_fonts)

def get_model_subsamples(data, name, dataset):
    points = []
    points.append(data[data["model"]==f"{name}_10000"].mean()[dataset])
    points.append(data[data["model"]==f"{name}_50000"].mean()[dataset])
    points.append(data[data["model"]==f"{name}_100000"].mean()[dataset])
    points.append(data[data["model"]==f"{name}"].mean()[dataset])
    return points

def plot_model_efficiency( metric):
    figsize = (4., 3.9)
    fig, ax = plt.subplots(1, 1, figsize=figsize)

    ax.plot(get_model_subsamples(df, "deen_base_ls", metric), label=r"Base + \textsc{LS}", linestyle='--', linewidth=1, marker='o', color='b')
#     ax.plot(get_model_subsamples(df, "deen_prior_3M_kl",metric), label=r"\textsc{LM}-prior 3M", linestyle='--', linewidth=1, marker='s', color='r')
    ax.plot(get_model_subsamples(df, "deen_prior_30M_kl",metric), label=r"Base + Prior", linestyle='--', linewidth=1, marker='P', color='g')
#     plt.plot(get_model_subsamples(df, "deen_prior_30M_kl_ls",metric), label='PRIOR(30)+LS', linestyle='--', linewidth=1, marker='P', color='r')
#     plt.title("Data efficiency")
    plt.xticks([0,1,2,3], ["10K", "50K", "100K", "full"])
#     plt.xticks([0,1,2], ["50K", "100K", "full (275K)"])
    plt.xlabel("parallel data")
    plt.ylabel("BLEU", labelpad=0.6)
#     Axes.set_ylabel(self, ylabel, fontdict=None, labelpad=None, **kwargs)[source]
    plt.legend(loc='upper left', handlelength=3)
    plt.margins(x=.025)
    plt.margins(y=.025)
#     plt.xlim(0, 6)
#     plt.ylim(0, 0.8)
    fig.tight_layout()

    plt.savefig(f'efficiency_{metric}.pdf', bbox_inches='tight', format="pdf",pad_inches = 0)



In [202]:
plot_model_efficiency("test")
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …